# Scatter plots with Plotly.NET

## Graph description
> A scatter plot (also called a scatterplot, scatter graph, scatter chart, scattergram, or scatter diagram) is a type of plot or mathematical diagram using Cartesian coordinates to display values for typically two variables for a set of data
_[wikipedia](https://en.wikipedia.org/wiki/Scatter_plot)_

Scatter plots are one of the most basic graphs imaginable. In their most basic form, they plot two variables on a 2D plane as points. The two coordinates on that plane are often referred to as pairs of _(X,Y)_ coordinate pairs, and the position scales are called _X and Y axes_.

## Graph data

let's create two sets of data for the _X_ and _Y_ Coordinates of our points respectively.

the values in `x` are linearily spaced between 0 and $2*\pi$, while the y coordinates will have the corresponding [sine]() values ($sin(x)$)

In [13]:
let x = [0. .. 0.1 .. (2.*System.Math.PI)]
let y = x |> List.map sin

## Reference Plotly.NET

In [14]:
#r "nuget: Plotly.NET.Interactive"

Installed Packages Plotly.NET.Interactive, 4.0.0

## A simple scatter plot

Plotly.NET offers many high-level abstractions for graphs with ist `Chart` API. 

`Chart.Scatter` creates a scatter plot. it needs at least `x` and `y` arguments, and a `mode` argument that defines how the points will look like.
For our first example, let's use `Mode.Markers` here to create simple points:

In [15]:
open Plotly.NET

Chart.Scatter(
    x = x,
    y = y,
    mode = StyleParam.Mode.Markers
)

<!-- Plotly chart will be drawn inside this DIV -->

## Using Chart.Point and some basic layout styling

`Chart.Point` is just `Chart.Scatter` with `mode` set to `markers`. If we just want to plot points, it is the method of choice. Additionally, lets use some chart styling functions:

In [16]:
Chart.Point(
    x = x,
    y = y,
    Name = "y = sin(x)",
    ShowLegend = true
)
|> Chart.withXAxisStyle(TitleText = "x")
|> Chart.withYAxisStyle(TitleText = "y")
|> Chart.withTitle ("f(x) = sin(x)")

<!-- Plotly chart will be drawn inside this DIV -->

## Connecting data with lines

When the order of our points has a meaning (for example, our `x` could be `time`, and `y` could be an observation we make in dependence of time), it can make sense to connect them via lines.

We could use `Mode.Lines` with `Chart.Scatter`, but there is also `Chart.Line` which does just that:

In [17]:
Chart.Line(
    x = x,
    y = y
)
|> Chart.withXAxisStyle(TitleText = "time [s]")
|> Chart.withYAxisStyle(TitleText = "some observation value")
|> Chart.withTitle ("Our observations")

<!-- Plotly chart will be drawn inside this DIV -->

## When not to use lines

As indicated above, connecting our data points via lines only makes sense if there is some inherent meaning to the connection, such as is indicating observations occuring in sequence.

Scatter plots can also be used to investigate the distribution of data points across two dimensions (often leading to a 'point cloud').

If we connect the points on such a plot where we have no onherent meaning to succession of values, it does not make too much sense.

Let's take a look at this side-by-side, using `Chart.Grid` for a multi-chart layout:

In [18]:
let rnd = new System.Random(69)
let rnd_x = [for i in 0 .. 200 -> rnd.NextDouble()]
let rnd_y = [for i in 0 .. 200 -> rnd.NextDouble()]

[
    Chart.Point(
        x = rnd_x,
        y = rnd_y,
        Name = "Point cloud"
    )
    |> Chart.withXAxisStyle(TitleText = "x")
    |> Chart.withYAxisStyle(TitleText = "y")
    Chart.Line(
        x = rnd_x,
        y = rnd_y,
        Name = "lines"
    )
    |> Chart.withXAxisStyle(TitleText = "x")
    |> Chart.withYAxisStyle(TitleText = "y")
]
|> Chart.Grid(nRows = 1, nCols = 2)
|> Chart.withSize(Width=1000,Height =600)
|> Chart.withTitle "sometimes, lines do not make sense"

<!-- Plotly chart will be drawn inside this DIV -->

## Bubble charts - introducing a third dimension to scatter plots

In the previous examples, all points are of the same size, and the size does not have any assiocated dimension.

**Bubble** charts change that by associating a third variable to the point size.

Imagine the following scenario:

We have two imaginary countries - `ALand` and `BLand`, which we want to compare across their `population` and [`GDP`](https://en.wikipedia.org/wiki/Gross_domestic_product) over time.

using `Chart.Bubble`, we can add the one dimension (here, i choose `GDP`) to the point size:

In [19]:
// A Land
let gdp_a = [20; 25; 35; 40; 50]
let population_a = [500; 530; 520; 500; 510]

//B Land
let gdp_b = [30; 30; 33; 31; 35]
let population_b = [400; 500; 600; 700; 800]

let time = [1900; 1910; 1920; 1930; 1940]

[
    Chart.Bubble(
        x = time,
        y = population_a,
        sizes = gdp_a,
        Name = "ALand",
        MultiText = gdp_a, // show gdp values as text in addition to bubble size
        TextPosition = StyleParam.TextPosition.Auto // set textposition to make gdp values visible
    )

    Chart.Bubble(
        x = time,
        y = population_b,
        sizes = gdp_b,
        Name = "BLand",
        MultiText = gdp_b, // show gdp values as text in addition to bubble size
        TextPosition = StyleParam.TextPosition.Auto  // set textposition to make gdp values visible
    )
]
|> Chart.combine
|> Chart.withXAxisStyle (TitleText = "Time [y]")
|> Chart.withYAxisStyle (TitleText = "Population [Million]")
|> Chart.withTitle "Comparison of ALand and BLand<br>regarding GDP and population growth over time"

<!-- Plotly chart will be drawn inside this DIV -->

What can we see here? 
- ALand has a pretty stagnant population size (y value), while it's GDP (bubble size) is increasing with time, meaning the wealth of individuals is rising over time.
- BLand in contrast has a rising population, while having a stagnant GDP, indicating that individual wealth is decreasing.

Since we have a `time` axis, we can also connect the bubbles via a lines to further emphasize the time evolution.

As `Chart.Bubble` has no arguments for that, we can fall back on using `Chart.Scatter` like this:



In [20]:
open Plotly.NET.TraceObjects

[
    Chart.Scatter(
        x = time,
        y = population_a,
        mode = StyleParam.Mode.Lines_Markers_Text,
        Name = "ALand",
        MultiText = gdp_a,
        TextPosition = StyleParam.TextPosition.Auto,
        Marker = Marker.init(MultiSize = gdp_a) // the marker object controls the style of the individual points
    )

    Chart.Scatter(
        x = time,
        y = population_b,
        mode = StyleParam.Mode.Lines_Markers_Text,
        Name = "BLand",
        MultiText = gdp_b,
        TextPosition = StyleParam.TextPosition.Auto,
        Marker = Marker.init(MultiSize = gdp_b) // the marker object controls the style of the individual points
    )
]
|> Chart.combine
|> Chart.withXAxisStyle (TitleText = "Time [y]")
|> Chart.withYAxisStyle (TitleText = "Population [Million]")
|> Chart.withTitle "Comparison of ALand and BLand<br>regarding GDP and population growth over time"

<!-- Plotly chart will be drawn inside this DIV -->

## Closing remarks

- In general, Scatter plots are used to visualize the relationship of 2 variables on a 2D plane. Examples include:
  - a timed sequence of observations, with then `x` dimension being `time` and the `y` dimension being any other variable which is observed over time.
  - a point cloud, giving a first look on how any kind of two-dimensional data is distributed

- Depending on the type of visualization, it can make sense to connect data points with lines.

- You can add another dimension to a scatter plot by changing the point size, leading to a Bubble chart.

- Plotly.NET offers several easy ways of creating different scatter plots, such as 
  - `Chart.Point`
  - `Chart.Line`
  - `Chart.Bubble`

## More Plotly.NET

Plotly.NET is a feature-rich graphing library for .NET programming languages. 

Check out the [source repository on github](https://github.com/plotly/Plotly.NET) and [in-depth F# docs](https.//plotly.net) !